In [1]:
import requests
import json
import os
import pandas as pd

In [8]:
def crawler_data(url,start,pagesize,info):
    '''
        参数url：需要爬取得网址的url
        参数start：该参数指定从那一页开始抓取
        参数pagesize：每一次抓取得页数
    '''
    post_param = {
              'start': start,
              'pageSize':pagesize,
              'cityId':'538',  # 该参数表示你所想搜寻的岗位的城市==>上海为538 北京为530 全国为489
              'salary':'0,0',  # 期望的薪资==> -1 表示不限
              'workExperience':'-1', # 工作经验 ==> -1 表示不限
              'companyType':'-1', # 公司性质 ==> -1 表示不限
              'employmentType':'-1', 
              'jobWelfareTag':'-1', 
              'kw':info, # 你想查找的岗位名称
              'kt':'3',
              '_v':'0.69816371',
              'x-zp-page-request-id': '625b805a75f2435ba4db425bd29b8d4f-1556416701269-989985'
             }
    # 设置headers防止反扒
    headers={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.109 Safari/537.36'
    }
    response=requests.get(url,params=post_param,headers=headers,verify = True) # 智联采用https协议，verify = True防止异常产生
    return response

In [15]:
url = "https://fe-api.zhaopin.com/c/i/sou" # 智联招聘的岗位信息使用的js加载，分析可得到其url
info = "深度学习" # 你想要查询的岗位名称
start = 0 # 起始页
pageSize = 100 # 每页所要拿取得招聘信息的数目
res =crawler_data(url,start,pageSize,info)

In [16]:
str_data=res.content.decode("utf8") 
json_data=json.loads(str_data) # 将json数据转换为python对象

In [18]:
if json_data["code"]==200:
    for i in range(pageSize):
        jobname = json_data["data"]["results"][i]["jobName"]
        salary = json_data["data"]["results"][i]["salary"]
        positionURL = json_data["data"]["results"][i]["positionURL"]
        company_name = json_data["data"]["results"][i]["company"]["name"]
        eduLevel = json_data["data"]["results"][i]["eduLevel"]["name"]
        workingExp = json_data["data"]["results"][i]["workingExp"]["name"]
        d ={
                '岗位名称':jobname,
                '薪资':salary,
                '详情页URL':positionURL,
                '公司名称':company_name,
                '学历':eduLevel,
                '工作经验':workingExp
        }
        df = pd.DataFrame(data=d,index=[i])    
        df.to_csv("列表页.csv",mode='a',header=None,index=None)